On charge notre dataset et on supprime les données ayant les valeurs null

In [1]:
import numpy as np
import random
import pandas as pd
import warnings
import os

warnings.filterwarnings('ignore')

train = pd.read_csv('../../data/tweet_train.csv')
train.dropna(inplace=True)
train

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


On nettoie nos données

In [2]:
import re
import string
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub(f'[{string.punctuation}]', '', text) #remove punctuation (list available in string.punctuation)
    text = re.sub('\n', '', text) #remove new line
    return text

train['original_text'] = train['text'].copy()
train['original_selected_text'] = train['selected_text'].copy()
train['text'] = train['original_text'].apply(clean_text) #apply cleaning
train['selected_text'] = train['original_selected_text'].apply(clean_text) #apply cleaning

train.head()

,textID,text,selected_text,sentiment,original_text,original_selected_text
0,cb774db0d1,id have responded if i were going,id have responded if i were going,neutral,"I`d have responded, if I were going","I`d have responded, if I were going"
1,549e992a42,sooo sad i will miss you here in san diego,sooo sad,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD
2,088c60f138,my boss is bullying me,bullying me,negative,my boss is bullying me...,bullying me
3,9642c003ef,what interview leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone
4,358bd9e861,sons of why couldnt they put them on the rel...,sons of,negative,"Sons of ****, why couldn`t they put them on t...","Sons of ****,"


Code qui permet de demander à l'utilisateur le tweet qu'il souhaite, le nombre de commentaires qu'il souhaite, et génère ainsi les commentaires en conséquence (attention, pour gagner du temps, veuillez entrer un petit nombre de commentaires).

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


tweet = input('Rentrer votre Tweet : ')
print(f'Tweet : {tweet}')
nbr_comment = int(input('Combien de commentaires voulez-vous générer ? '))

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

messages = [
    {"role": "system", "content": f"Generate exactly {nbr_comment} distinct and independent Twitter comments representing {nbr_comment} user reactions. Each comment should reflect the viewpoint of the tweet in question, adapt to the written language, stay within the 280-character limit, and appear realistic for a Twitter comment where each comment must correspond to a reaction that a user could have when reading this tweet. Returns comments in this form: '1. first comment\n\n2. Second Comment\n\n....'"},
    {"role": "user", "content": tweet},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)

print(output[0]['generated_text'])
liste = output[0]['generated_text'].split('\n')
comments = []
for line in liste:
    if line.strip() != "":
        separator_index = line.find(". ")
        if separator_index != -1:
            index = separator_index + 2
            comments.append(line[index:].replace('"', ''))
        else:
            comments.append(line.replace('"', ''))
comments = comments[:nbr_comment]
comments

2024-10-04 15:44:50.977057: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 15:44:51.014751: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-04 15:44:51.758712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tweet : I think Sandro would be the dream intern for your company!


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.72it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are not running the flash-attention implementation, expect numerical differences.


 1. Absolutely agree! Sandro's skills are a perfect match for our team. #DreamIntern

2. Sandro's passion for the field is inspiring. He'd be a great addition to our team!

3. I've seen Sandro's work, and he's exactly the kind of talent we need. #InternGoals

4. Sandro's dedication to learning is commendable. He'd be a valuable asset to our company.

5. Sandro's innovative approach to problem-solving would bring a fresh perspective to our team.


["Absolutely agree! Sandro's skills are a perfect match for our team. #DreamIntern",
 "Sandro's passion for the field is inspiring. He'd be a great addition to our team!",
 "I've seen Sandro's work, and he's exactly the kind of talent we need. #InternGoals",
 "Sandro's dedication to learning is commendable. He'd be a valuable asset to our company.",
 "Sandro's innovative approach to problem-solving would bring a fresh perspective to our team."]

Dans le but d'évaluer notre modèle, nous allons faire la même chose sur 10 tweets de notre dataset pris aléatoirement (10, car cela prendrait trop de temps sinon).

In [ ]:
sample_values = train['original_text'].sample(n=10)

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
tweets = []

for tweet in sample_values:
    print(tweet)
    nbr_comment = 15

    messages = [
        {"role": "system", "content": f"Generate exactly {nbr_comment} distinct and independent Twitter comments representing {nbr_comment} user reactions. Each comment should reflect the viewpoint of the tweet in question, adapt to the written language, stay within the 280-character limit, and appear realistic for a Twitter comment where each comment must correspond to a reaction that a user could have when reading this tweet. Returns comments in this form: '1. first comment\n\n2. Second Comment\n\n....'"},
        {"role": "user", "content": tweet},
    ]

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )

    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }

    output = pipe(messages, **generation_args)
    print(output[0]['generated_text'])
    liste = output[0]['generated_text'].split('\n')
    comments = []
    for line in liste:
        if line.strip() != "":
            separator_index = line.find(". ")
            if separator_index != -1:
                index = separator_index + 2
                comments.append(line[index:].replace('"', ''))
            else:
                comments.append(line.replace('"', ''))
    comments = comments[:nbr_comment]
    phrase = comments + [tweet] # On ajoute le tweet à la fin
    print(comments)
    tweets.append(phrase)

 if I could make it to Philadelphia this year I would be there!! but sadly, i cannot
 1. Wishing you the best for your trip to Philadelphia! Hope you get there soon.

2. So sorry to hear that you can't make it. Philadelphia is a great city!

3. Philadelphia is on my bucket list too! Hope you can visit next time.

4. That's a bummer. Philadelphia has so much to offer.

5. I'm rooting for you to get there this year. Philadelphia is amazing!

6. I've been there once, it's a beautiful city. Hope you can make it next time.

7. Philadelphia is a must-visit. I hope you get the chance to see it.

8. That's a shame. I've always wanted to visit Philadelphia.

9. I'm sorry to hear that. Philadelphia is a fantastic city to explore.

10. I'm sure you'll get there one day. Philadelphia is worth the wait.

11. Philadelphia is a city with so much history and culture. Hope you can visit soon.

12. I'm sorry to hear that. Philadelphia is a great city to visit.

13. I've been there and it's a beautiful c

Nous allons maintenant, pour chaque tweet, afficher son sentiment initial et, pour chacun de ses commentaires, analyser son sentiment ainsi que son sujet principal.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('vader_lexicon')

# Fonction pour extraire le sujet principal de la phrase
def extract_subject(sentence):
    tokens = word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    subjects = [word for word, tag in tagged if tag.startswith('NN')]
    return subjects

# Fonction pour déterminer le sentiment de la phrase
def analyze_sentiment(sentence):
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(sentence)['compound']
    if sentiment_score > 0.05:
        return 'positive'
    elif sentiment_score < -0.05:
        return 'negative'
    else:
        return 'neutral'

for phrase in tweets:
    print('\n\nTweet:', phrase[-1], '   Sentiment :', train.loc[train['original_text'] == phrase[-1]].iloc[0]['sentiment'])
    for p in phrase :
        subjects = extract_subject(p)
        sentiment = analyze_sentiment(p)
        print("Sujet principal :", subjects)
        print("Sentiment :", sentiment)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...




Tweet: is watching wolverine at last.    Sentiment : neutral
Sujet principal : ['Ca', 'thoughts', 'movie', '🦾']
Sentiment : neutral
Sujet principal : ['Logan', 'action', '🎬']
Sentiment : neutral
Sujet principal : ['Wolverine', 'fan', 'favorite', 'ages', '😁']
Sentiment : positive
Sujet principal : ['Wolverine', 'movie', 'comics', '📚']
Sentiment : positive
Sujet principal : ['Time', 'Logan', 'story', 'unfolds', 'screen', '🍿']
Sentiment : neutral
Sujet principal : ['Wolverine', 'movie', '💥']
Sentiment : positive
Sujet principal : ['series', 'blast', 'Ca', 'installment', '🌟']
Sentiment : neutral
Sujet principal : ['Logan', 'journey', 'filmmakers', '🎥']
Sentiment : neutral
Sujet principal : ['Wolverine', 'resilience', 'Time', 'action', '💪']
Sentiment : neutral
Sujet principal : ['fan', 'comics', 'movie', 'lore', '📖']
Sentiment : positive
Sujet principal : ['action', 'scenes', 'Wolverine', '🚀']
Sentiment : positive
Sujet principal : ['Logan', 'character', 'adaptation', '🤔']
Sentiment : pos

En somme, nous remarquons que les sujets sont similaires à ceux du tweet à chaque fois, et que les sentiments des commentaires sont en moyenne identiques à ceux du tweet.